In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig

path = "training/pretrain/compile/0.0002/"
tokenizer_path = "training/tokenizer/2048"
state_dict = torch.load(path + "hf_model.pt", weights_only=False)
state_dict = {k.removeprefix('model.'): v for k, v in state_dict['state']['model'].items()}

model_name = "roneneldan/TinyStories-33M"
config = AutoConfig.from_pretrained(model_name)
model = AutoModelForCausalLM.from_config(config=config)
model.resize_token_embeddings(2048)
model.load_state_dict(state_dict)
model.eval()
model.to("cuda")
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

config.json:   0%|          | 0.00/968 [00:00<?, ?B/s]

In [2]:
sum(p.numel() for p in model.parameters())

31489536

In [3]:
tokenizer

GPT2TokenizerFast(name_or_path='training/tokenizer/2048', vocab_size=2048, model_max_length=2048, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}
)

In [7]:
inp = "Once upon a time,"
inp = tokenizer(inp, return_tensors="pt").to(model.device)
out = model.generate(**inp, max_length=500, do_sample=True, top_k=50, top_p=0.95, temperature=0.7)
batch = tokenizer.batch_decode(out, skip_special_tokens=True)
print(batch[0])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Once upon a time, there was a little girl named Lily. She loved to play outside in the sun. One day, she saw a big, ugly bug on the ground. She was scared, but her mommy said it was okay.

Lily went to her mommy and said, "Mommy, look! A ugly bug is stuck in the ugly bug." Her mommy looked up and said, "Don't worry, Lily. The bug is just trying to get free."

Lily took a deep breath and sat down on the grass. She felt so happy to be free. She said to her mommy, "Look, mommy! I saw a bug! It was so pretty." Her mommy smiled and said, "Yes, it's a pretty bug. We can still have fun in the ugly bug."


In [10]:
out.shape

torch.Size([1, 500])

In [11]:
import pandas as pd
import torch


# Read the evaluation prompts from CSV file
eval_prompts = pd.read_csv("evaluation_prompts.csv")

# Display the first few rows to understand the data structure
print(f"Number of evaluation prompts: {len(eval_prompts)}")
print(eval_prompts.head())

# Generate text for each prompt
results = []
for i, prompt in enumerate(eval_prompts['prompt'] if 'prompt' in eval_prompts.columns else eval_prompts.iloc[:, 0]):
    # Tokenize the prompt
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    # Generate text
    with torch.no_grad():
        outputs = model.generate(
            **inputs, 
            max_length=300, 
            do_sample=True, 
            
            top_k=50, 
            top_p=0.95, 
            temperature=0.7
        )
    
    # Decode the generated text
    generated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0].removeprefix(prompt).strip()
    
    # Store the result
    results.append({
        'prompt': prompt,
        'completion': generated_text
    })
    
    # Print progress
    if i % 10 == 0:
        print(f"Processed {i+1}/{len(eval_prompts)} prompts")

# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Display a sample of results
print("\nSample of generated texts:")
print(results_df.head(3))

# Save results to CSV
results_df.to_csv("generation_results.csv", index=False)
print("\nResults saved to generation_results.csv")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Number of evaluation prompts: 300
                                              prompt  completion
0  Once upon a time, there lived a bunny in a fie...         NaN
1  Once upon a time, there lived a bunny in a fie...         NaN
2  Once upon a time, there lived a bunny in a fie...         NaN
3  Once upon a time, there lived a bunny in a fie...         NaN
4  Once upon a time, there lived a bunny in a fie...         NaN


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Processed 1/300 prompts


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f3fe15be4e0>>
Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


KeyboardInterrupt: 